<a href="https://colab.research.google.com/github/ArturAzarskyy/CSC413-Stock-Prediction/blob/main/Transformer/transformer_prepros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is the preprocessing code for our modles

Note that some parts of ideas and code was taken form the Jan Schmitz notebook on IBM stock prediciton [IBM stock predictor](https://github.com/JanSchm/CapMarket/blob/master/bot_experiments/IBM_Transformer%2BTimeEmbedding.ipynb). Though Jan S. worked only with one stock we extended the idea to multiple stocks as well as used different dataset. We are also looking at a bit different model as well.


In [1]:
!echo '{"username":"arturusmaximus","key":"4f14194978499e9ae1ad6adb74b94add"}' > /content/kaggle.json
!cp kaggle.json ~/.kaggle/
# !kaggle datasets download -d borismarjanovic/price-volume-data-for-all-us-stocks-etfs
!kaggle datasets download -d qks1lver/amex-nyse-nasdaq-stock-histories
# !unzip price-volume-data-for-all-us-stocks-etfs.zip
!unzip amex-nyse-nasdaq-stock-histories.zip

amex-nyse-nasdaq-stock-histories.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  amex-nyse-nasdaq-stock-histories.zip
replace all_symbols.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [11]:
import pandas as pd
import torch as ty
import os.path
import numpy as np

time_step = 3
stock_histoy_lenght = 128 # this does not include the label
# stock_histoy_lenght = 63 # this does not include the label
f_train_data   = open("f_train_data.npy", 'ab')
f_train_labels = open("f_train_labels.npy", 'ab')
f_val_data     = open("f_val_data.npy", 'ab')
f_val_labels   = open("f_val_labels.npy", 'ab')
f_test_data    = open("f_test_data.npy", 'ab')
f_test_labels  = open("f_test_labels.npy", 'ab')


In [12]:
import pdb
# train_data = None
# train_labels = None
# val_data = None
# val_labels = None
# test_data = None
# test_labels = None

with open('all_symbols.txt') as topo_file:
    for line in topo_file:
        if os.path.isfile("full_history/"+line[:-1]+".csv"):
            df = pd.read_csv("full_history/"+line[:-1]+".csv",
                             delimiter=',', 
                             usecols=['date', 'open', 'high', 'low', 'close', 'volume'])
            
            df['volume'].replace(to_replace=0, method='ffill', inplace=True) 
            df['open'] = df['open'].pct_change()
            df['high'] = df['high'].pct_change()
            df['low'] = df['low'].pct_change()
            df['close'] = df['close'].pct_change()
            df['volume'] = df['volume'].pct_change()

            df.dropna(how='any', axis=0, inplace=True)
            if len(df.index.values) == 0:
                continue
            df.sort_values('date', inplace=True)
            df = df.reset_index(drop=True)
            valid_start = sorted(df.index.values)[-int(0.3*df.shape[0])]  
            test_start = sorted(df.index.values)[-int(0.1*df.shape[0])]

            if int(0.2*df.shape[0]) < stock_histoy_lenght+1 or int(0.1*df.shape[0]) < stock_histoy_lenght+1:
                continue

            # Normalizing the data

            min_return = min(df[(df.index < valid_start)][['open', 'high', 'low', 'close']].min(axis=0))
            max_return = max(df[(df.index < valid_start)][['open', 'high', 'low', 'close']].max(axis=0))

            df['open'] = (df['open'] - min_return) / (max_return - min_return)
            df['high'] = (df['high'] - min_return) / (max_return - min_return)
            df['low'] =  (df['low'] - min_return) / (max_return - min_return)
            df['close']= (df['close'] - min_return) / (max_return - min_return)

            min_volume = df[(df.index < valid_start)]['volume'].min(axis=0)
            max_volume = df[(df.index < valid_start)]['volume'].max(axis=0)

            df['volume'] = (df['volume'] - min_volume) / (max_volume - min_volume)


            df.drop(columns=['date'], inplace=True)
            df_train = df[(df.index < valid_start)]
            df_val = df[(df.index >= valid_start) & (df.index < test_start)]
            df_test = df[(df.index >= test_start)]
            update_train_data = []
            update_train_labels = []
            update_val_data = []
            update_val_labels = []
            update_test_data = []
            update_test_labels = []


            c_train_data = df_train.values
            c_val_data = df_val.values
            c_test_data = df_test.values
            for i in range(stock_histoy_lenght, len(c_train_data), time_step):
                update_train_data.append(c_train_data[i-stock_histoy_lenght:i])
                update_train_labels.append(c_train_data[i, 3])
                # train_data.append(c_train_data[i-stock_histoy_lenght:i])
                # train_labels.append(c_train_data[i, 3])
            for i in range(stock_histoy_lenght, len(c_val_data), time_step):
                update_val_data.append(c_val_data[i-stock_histoy_lenght:i])
                update_val_labels.append(c_val_data[i, 3])
                # val_data.append(c_val_data[i-stock_histoy_lenght:i])
                # val_labels.append(c_val_data[i, 3])
            for i in range(stock_histoy_lenght, len(c_test_data), time_step):
                update_test_data.append(c_test_data[i-stock_histoy_lenght:i])
                update_test_labels.append(c_test_data[i, 3])
                # test_data.append(c_test_data[i-stock_histoy_lenght:i])
                # test_labels.append(c_test_data[i, 3])
            np.save(f_train_data,np.array(update_train_data))
            np.save(f_train_labels,np.array(update_train_labels))
            np.save(f_val_data, np.array(update_val_data))
            np.save(f_val_labels, np.array(update_val_labels))
            np.save(f_test_data, np.array(update_test_data))
            np.save(f_test_labels, np.array(update_test_labels))
            # if train_data is None:
            #     train_data = np.array(update_train_data)
            #     train_labels = np.array(update_train_labels)
            # else:
            #     train_data = np.concatenate((train_data, update_train_data), axis=0)
            #     train_labels = np.concatenate((train_labels, update_train_labels), axis=0)

            # if val_data is None:
            #     val_data =  np.array(update_val_data)
            #     val_labels = np.array(update_val_labels)
            # else:
            #     val_data = np.concatenate((val_data, update_val_data), axis=0)
            #     val_labels = np.concatenate((val_labels, update_val_labels), axis=0)

            # if test_data is None:
            #     test_data = np.array(update_test_data)
            #     test_labels = np.array(update_test_labels)
            # else:
            #     test_data = np.concatenate((test_data, update_test_data), axis=0)
            #     test_labels = np.concatenate((test_labels, update_test_labels), axis=0)



In [13]:
f_train_data.close()
f_train_labels.close()
f_val_data.close()
f_val_labels.close()
f_test_data.close()
f_test_labels.close()

In [14]:
train_data   = np.load("f_train_data.npy", mmap_mode='r')
train_labels = np.load("f_train_labels.npy", mmap_mode='r')
val_data     = np.load("f_val_data.npy", mmap_mode='r')
val_labels   = np.load("f_val_labels.npy", mmap_mode='r')
test_data    = np.load("f_test_data.npy", mmap_mode='r')
test_labels  = np.load("f_test_labels.npy", mmap_mode='r')

In [16]:
train_data.shape

(472, 128, 5)

In [ ]:
train_data = np.array(train_data)
train_labels = np.array(train_labels)

In [1]:
train_data, train_labels = np.array(train_data), np.array(train_labels)

NameError: ignored

In [1]:
train_data, train_labels = np.array(train_data), np.array(train_labels)
val_data, val_labels = np.array(val_data), np.array(val_labels)
test_data, test_labels = np.array(test_data), np.array(test_labels)

NameError: ignored

In [ ]:

np.save("stocks_train_data_no_m_avg", train_data)
np.save("stocks_train_labels_no_m_avg", train_labels)
np.save("stocks_val_data_no_m_avg", val_data)
np.save("stocks_val_labels_no_m_avg", val_labels)
np.save("stocks_test_data_no_m_avg", test_data)
np.save("stocks_test_labels_no_m_avg", test_labels)